<a href="https://colab.research.google.com/github/BaRRdAk/rcnn/blob/main/Mask_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Installation

In [ ]:
%tensorflow_version 1.x
!pip install --upgrade h5py==2.10.0
!git clone https://github.com/pysource7/Mask_RCNN
import sys
sys.path.append("/content/Mask_RCNN/demo/")
from train_mask_rcnn_demo import *
%matplotlib inline


In [ ]:
!nvidia-smi

# 2. Image Dataset

In [ ]:
# Extract Images
images_path = "images.zip"
annotations_path = "annotations.json"

extract_images(os.path.join("/content/", images_path), "/content/dataset")

In [ ]:
dataset_train = load_image_dataset(os.path.join("/content/", annotations_path), "/content/dataset", "train")
dataset_val = load_image_dataset(os.path.join("/content/", annotations_path), "/content/dataset", "val")
print('Validation: %d' % len(dataset_val.image_ids))

class_number = dataset_train.count_classes()
print("Classes: {}".format(class_number))

In [ ]:
# Load image samples
display_image_samples(dataset_train)

In [ ]:
# Load Configuration
config = CustomConfig(class_number)
#config.display()
model = load_training_model(config)

# 3. Training

Train Mask RCNN on your custom Dataset

In [ ]:
# Start Training
# This operation might take a long time
train_head(model, dataset_train, dataset_train, config)

# 4. Detection (test your model on a random image)

In [ ]:
# Load Test Model
# The latest trained model will be loaded
test_model, inference_config = load_test_model(class_number)

In [ ]:
# Test on a random image
test_random_image(test_model, dataset_val, inference_config)

# 5. Run Mas-RCNN on Images

In [ ]:
test_model, inference_config = load_inference_model(1, "/content/drive/MyDrive/RCNN/mask_rcnn_object_0005-2.h5")

In [ ]:
# Load Image
img = cv2.imread("/content/drive/MyDrive/RCNN/scr_15.png")

image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect result
r = test_model.detect([image])[0]

In [ ]:
from mrcnn.visualize import random_colors, get_mask_contours, draw_mask
from google.colab.patches import cv2_imshow

# Get Coordinates and show it on the image
object_count = len(r["class_ids"])
colors = random_colors(object_count)
for i in range(object_count):
  # 1. Mask
  mask = r["masks"][:, :, i]

  contours = get_mask_contours(mask)
  for cnt in contours:
    cv2.polylines(img, [cnt], True, colors[i], 2)
    vimg = draw_mask(img, [cnt], colors[i])

cv2_imshow(img)

# 6. Vectorize mask

In [ ]:
from osgeo import osr, gdal, ogr

bottom_left = [4611314.9914456,6590402.9860494]
top_right = [4655320.4908912,6632051.0480247]
top_left_coord = [top_right[0], bottom_left[1]]

ds = gdal.Open("/content/drive/MyDrive/RCNN/scr_15.png")
width = ds.RasterXSize
height = ds.RasterYSize
x_pixel_res = height/(top_right[0] - bottom_left[0])
y_pixel_res = width/(top_right[1] - bottom_left[1])

print("x_pixel_res", x_pixel_res)
print("y_pixel_res", y_pixel_res)

multipolygon = ogr.Geometry(ogr.wkbMultiPolygon)

def convert_coord(x_pixel_res, y_pixel_res, top_left_coord, raster_coord):
  x = top_left_coord[0] + raster_coord[0]*x_pixel_res
  y = top_left_coord[1] - raster_coord[1]*y_pixel_res
  # print("raster_coord", raster_coord)
  # print("global_coord", [x,y])
  return [x,y]

for i in range(object_count):
  # 1. Mask
  mask = r["masks"][:, :, i]

  contours = get_mask_contours(mask)
  for cnt in contours:
    print(len(cnt))
    ring = ogr.Geometry(ogr.wkbLinearRing)
    for point in cnt:
      coord_global = convert_coord(x_pixel_res, y_pixel_res, top_left_coord, point)
      ring.AddPoint(coord_global[0], coord_global[1])
    end = convert_coord(x_pixel_res, y_pixel_res, top_left_coord, cnt[0])
    ring.AddPoint(end[0], end[1])
    polygon = ogr.Geometry(ogr.wkbPolygon)
    polygon.AddGeometry(ring)
    multipolygon.AddGeometry(polygon)

simpleMultipolygon = multipolygon.Simplify(0.2)


dst_layername = "POLYGONIZED_STUFF"
shapeDriver = ogr.GetDriverByName("ESRI Shapefile")
dst_ds = shapeDriver.CreateDataSource(dst_layername + ".shp")

# create the spatial reference system, WGS84
srs = osr.SpatialReference()
# srs.ImportFromEPSG(4326)
srs.ImportFromProj4("+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext +no_defs")

# create one layer 
dst_layer = dst_ds.CreateLayer(dst_layername, srs, ogr.wkbMultiPolygon)

# Add an ID field
idField = ogr.FieldDefn("id", ogr.OFTInteger)
dst_layer.CreateField(idField)

# Add an NAME field
dst_layer.CreateField(ogr.FieldDefn("name", ogr.OFTString))

# Add an SHORTNAME field
dst_layer.CreateField(ogr.FieldDefn("shortname", ogr.OFTString))

# Add an OKTMO field
dst_layer.CreateField(ogr.FieldDefn("oktmo", ogr.OFTString))

# Add an GUID field
dst_layer.CreateField(ogr.FieldDefn("guid", ogr.OFTString))

# Add an GRO field
dst_layer.CreateField(ogr.FieldDefn("gro", ogr.OFTString))


# Create the feature and set values
featureDefn = dst_layer.GetLayerDefn()
feature = ogr.Feature(featureDefn)
feature.SetGeometry(simpleMultipolygon)
feature.SetField("id", 1)
dst_layer.CreateFeature(feature)

feature = None

# Save and close DataSource
dst_ds = None

# geoTiffDriver = gdal.GetDriverByName("GTiff")
# target_ds = geoTiffDriver.Create("rasterize.tif", 500, 500, gdal.GDT_Byte)

# gdal.RasterizeLayer(target_ds, [3,2,1], dst_layer, burn_values=[200,220,240])

# 7. Merge shapefiles

In [ ]:
from osgeo import osr, gdal, ogr
import os

outputMergefn = 'merge.shp'
directory = "/Users/UserName/Downloads/"
fileStartsWith = 'test'
fileEndsWith = '.shp'
driverName = 'ESRI Shapefile'
geometryType = ogr.wkbMultiPolygon

out_driver = ogr.GetDriverByName( driverName )
if os.path.exists(outputMergefn):
    out_driver.DeleteDataSource(outputMergefn)
out_ds = out_driver.CreateDataSource(outputMergefn)
out_layer = out_ds.CreateLayer(outputMergefn, geom_type=geometryType)

fileList = os.listdir(directory)

for file in fileList:
    if file.startswith(fileStartsWith) and file.endswith(fileEndsWith):
        print file
        ds = ogr.Open(directory+file)
        lyr = ds.GetLayer()
        for feat in lyr:
            out_feat = ogr.Feature(out_layer.GetLayerDefn())
            out_feat.SetGeometry(feat.GetGeometryRef().Clone())
            out_layer.CreateFeature(out_feat)
            out_feat = None
            out_layer.SyncToDisk()